In [ ]:
# default_exp readproc

# readproc

This module can be used to generate '.csv' files containing counts for each read in a '.fasta' file.

> API details



In [ ]:
#hide
from nbdev.showdoc import *



In [ ]:
#export
import re
from pathlib import Path
import pandas as pd


def trim_cterm(inputFile, outputFile, motifToTrim):
    with open(inputFile, 'r') as inputFH:
        with open(outputFile,'w') as outputFH:
            i = 0
            for line in inputFH:
                i += 1
                if i%2 ==1: outputFH.write(line)
                else:
                    line = re.sub(motifToTrim, '', line)
                    #line += '\n'
                    outputFH.write(line)

                    
def fasta_to_counts(inputFile):
    
    
    df_seq_counts = pd.DataFrame()
    cur_name = ''
    cur_seq = ''
    dict_temp_seqs = {}
    i = 0
    #make a temporary dict for counting seqs
    with open(inputFile, 'r') as inputFH:
        for line in inputFH:
            print(i)
            if line[0] == '>': # lines starting with '>' are seq names 
                if cur_name != '': #add previous seq to dict, if it exists
                    if cur_name in dict_temp_seqs.keys():
                        dict_temp_seqs[cur_name][1] += 1 # add 1 to count for cur_seq
                    else:
                        dict_temp_seqs[cur_name] = [cur_seq, 1]
                cur_name = line.rstrip()[1:] # strip newline and '>'
                cur_seq = ''
                i += 1
            else: # odd lines are the actual sequences, concatenate until a new sequence starts
                cur_seq += line.rstrip()
                print(cur_seq)
        if cur_name in dict_temp_seqs.keys(): #add the last seq to the dict
            dict_temp_seqs[cur_name][1] += 1
        else:
            dict_temp_seqs[cur_name] =  [cur_seq, 1]
            
                
    return dict_temp_seqs

def dict_to_df:


def df_to_csv:

    
    #make a second dic with the final counts and also the sequences with terminal Zs, to build the DF
test_count_input_path = Path.cwd()/'tests/test-count-in.fasta' 
fasta_to_counts(test_count_input_path)

0
1
CCC
1
2
CCAC
2
3
CCC
3
4
CCAC
4
5
CCAC


{'123': ['CCC', 2], '128378': ['CCAC', 3]}

In [ ]:
data = {1: ['CCG', 1, 'ZCCGZ', 'abc'], 2: ['GTN', 3, 'ZCCGZ', 'cbdg']}

df_test = pd.DataFrame.from_dict(data, orient='index', columns=['seq', 'count', 'seqZ', 'seqname'])
df_test
if 3 in data.keys(): print('yes')
else: print('no')
data[1][1] +=1
data

no


{1: ['CCG', 2, 'ZCCGZ', 'abc'], 2: ['GTN', 3, 'ZCCGZ', 'cbdg']}

In [ ]:
line = '>abc \n'
seq = line.rstrip()[1:]
print(line)
print(seq)
line[0]
cur_seq += 'ab'
cur_seq

>abc 

abc


'abababab'

Testing

In [ ]:
test_input_path = Path.cwd()/'tests/test-trim-in.fasta'
test_trimmed_output_path = Path.cwd()/'tests/test-trim-out.fasta'
test_cterm_pattern_to_trim = 'LNEAIP.*$'

trim_cterm(test_input_path, test_trimmed_output_path, test_cterm_pattern_to_trim)

with open(test_trimmed_output_path, 'r') as test_FH:
    for i,line in enumerate(test_FH):
        if i == 0:
            assert line.rstrip() == '>123'
        if i == 1:
            assert line.rstrip() == 'CCC'
        if i == 2:
            assert line.rstrip() == '>128378'
        if i == 3:
            assert line.rstrip() == 'CCAC'
    
    assert i == 3

AssertionError: 